In [17]:
import pandas as pd
import os
from pprint import PrettyPrinter

In [18]:
os.getcwd()

'/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/notebooks'

In [19]:
pp = PrettyPrinter()

# Primary data sample

In [20]:
def engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    """Apply manual feature engineering transformations.

    Args:
        df (pd.DataFrame): data frame with raw features

    Returns:
        pd.DataFrame: data frame after feature engineering
    """
    # logger.info("Applying manual feature engineering transformations...")

    df["c_weekday"] = pd.to_datetime(df["i_visit_start_time"], unit="us").dt.weekday
    df["c_visit_start_hour"] = pd.to_datetime(
        df["i_visit_start_time"], unit="us"
    ).dt.hour

    return df

In [21]:
df = pd.read_csv('../data/03_primary/df_train.csv')

In [22]:
dfs = df.sample(20, random_state=22)
dfs

,i_full_visitor_id,i_visit_start_time,i_date,c_device_category,c_is_mobile,c_operating_system,c_browser,c_country,c_city,c_traffic_source,c_traffic_medium,c_traffic_campaign,c_is_first_visit,n_product_pages_viewed,n_total_hits,n_total_pageviews,n_total_visits,n_total_time_on_site,y_added_to_cart
2077,5.446260e+07,1.611670e+15,20210126,mobile,1,iOS,Safari,Taiwan,(not set),<Other>,referral,NaN,0,0,39,11,6,16613.0,0
9207,4.865536e+07,1.611713e+15,20210127,mobile,1,Web,Chrome,United States,Tampa,<Other>,<Other>,NaN,0,0,10,4,2,4589.0,0
6755,8.734718e+07,NaN,20210127,desktop,0,Web,Chrome,United States,(not set),(direct),(none),NaN,0,1,2,1,1,NaN,0
6704,5.908213e+07,1.611722e+15,20210127,desktop,0,Web,Chrome,Germany,Norderstedt,shop.googlemerchandisestore.com,referral,NaN,0,0,27,12,3,5076.0,0
9528,8.227701e+07,1.611646e+15,20210126,mobile,1,iOS,Safari,United States,(not set),(direct),(none),NaN,1,0,7,2,2,168.0,0
7421,4.523350e+06,1.611704e+15,20210126,desktop,0,Web,Chrome,United States,(not set),google,organic,NaN,0,0,37,18,3,13776.0,0
2578,9.699601e+06,1.611831e+15,20210128,desktop,0,Web,Chrome,India,(not set),google,organic,NaN,1,0,8,3,1,3887.0,0
9479,4.025593e+07,1.611628e+15,20210126,desktop,0,Web,Chrome,United States,Los Altos,google,organic,NaN,1,0,36,14,5,13298.0,0
2176,3.107230e+06,1.611853e+15,20210128,mobile,1,Web,Chrome,United States,(not set),(direct),(none),NaN,1,1,10,3,1,9408.0,0
8572,7.227836e+06,1.611621e+15,20210126,mobile,1,Web,Chrome,Russia,(not set),(direct),(none),NaN,1,1,5,2,1,25.0,0


## Engineer features

In [23]:
rows_to_select = list(range(3))
rows_to_select

[0, 1, 2]

In [24]:
columns_to_select = [
    "i_full_visitor_id",
    "i_visit_start_time"
]
columns_to_select

['i_full_visitor_id', 'i_visit_start_time']

In [25]:
df_sample = dfs.iloc[rows_to_select, :][columns_to_select]
df_sample

,i_full_visitor_id,i_visit_start_time
2077,5.446260e+07,1.611670e+15
9207,4.865536e+07,1.611713e+15
6755,8.734718e+07,NaN


In [26]:
dict_sample = df_sample.to_dict(orient="list")

In [27]:
pp.pprint(dict_sample)

{'i_full_visitor_id': [54462597.10306885, 48655355.20525688, 87347178.97957142],
 'i_visit_start_time': [1611669995733496.0, 1611713218266340.0, nan]}


In [28]:
df_processed = engineer_features(df_sample)
df_processed

,i_full_visitor_id,i_visit_start_time,c_weekday,c_visit_start_hour
2077,5.446260e+07,1.611670e+15,1.0,14.0
9207,4.865536e+07,1.611713e+15,2.0,2.0
6755,8.734718e+07,NaN,NaN,NaN


In [29]:
dict_processed = df_processed.to_dict(orient="list")

In [30]:
pp.pprint(dict_processed)

{'c_visit_start_hour': [14.0, 2.0, nan],
 'c_weekday': [1.0, 2.0, nan],
 'i_full_visitor_id': [54462597.10306885, 48655355.20525688, 87347178.97957142],
 'i_visit_start_time': [1611669995733496.0, 1611713218266340.0, nan]}


In [31]:
df_expected = pd.DataFrame({
        'c_visit_start_hour': [0, 7, 11],
        'c_weekday': [1, 3, 1],
        'i_full_visitor_id': [3033910.355860057, 81793309.0616803, 4293031.296243032],
        'i_visit_start_time': [1611619614341157.0,
                                1611820704267587.0,
                                1611661585573344.0]
    })

In [32]:
df_processed

,i_full_visitor_id,i_visit_start_time,c_weekday,c_visit_start_hour
2077,5.446260e+07,1.611670e+15,1.0,14.0
9207,4.865536e+07,1.611713e+15,2.0,2.0
6755,8.734718e+07,NaN,NaN,NaN


In [33]:
df_expected

,c_visit_start_hour,c_weekday,i_full_visitor_id,i_visit_start_time
0,0,1,3.033910e+06,1.611620e+15
1,7,3,8.179331e+07,1.611821e+15
2,11,1,4.293031e+06,1.611662e+15


In [34]:
df_processed.equals(df_expected)

False

In [35]:
df_processed.sort_index(axis=1).equals(df_expected.sort_index(axis=1))

False

## Data preparation utils

In [36]:
df = pd.DataFrame(
    {
        "c_first_good_colname": ["cat1", "cat2", "cat1"],
        "n_second_good_colname": [1, 3, 5],
        "c_first_bad_colname": [7, 8, 9],
        "n_second_bad_colname": ["3.0", "2.2", "5.6"],
    }
)

In [37]:
df

,c_first_good_colname,n_second_good_colname,c_first_bad_colname,n_second_bad_colname
0,cat1,1,7,3.0
1,cat2,3,8,2.2
2,cat1,5,9,5.6


In [38]:
df.dtypes

c_first_good_colname     object
n_second_good_colname     int64
c_first_bad_colname       int64
n_second_bad_colname     object
dtype: object

In [39]:
from ga4_mlops.pipelines.data_preparation_utils import extract_column_names, ensure_column_types

In [40]:
_, num_cols, cat_cols, _ = extract_column_names(df)

In [41]:
df2 = ensure_column_types(df, num_cols, cat_cols)

In [42]:
df2

,c_first_good_colname,n_second_good_colname,c_first_bad_colname,n_second_bad_colname
0,cat1,1,7,3.0
1,cat2,3,8,2.2
2,cat1,5,9,5.6


In [43]:
df2.dtypes

c_first_good_colname     category
n_second_good_colname       int64
c_first_bad_colname      category
n_second_bad_colname      float64
dtype: object

In [44]:
list(df2.dtypes)

[CategoricalDtype(categories=['cat1', 'cat2'], ordered=False),
 dtype('int64'),
 CategoricalDtype(categories=[7, 8, 9], ordered=False),
 dtype('float64')]

In [45]:
df2.dtypes.to_list()

[CategoricalDtype(categories=['cat1', 'cat2'], ordered=False),
 dtype('int64'),
 CategoricalDtype(categories=[7, 8, 9], ordered=False),
 dtype('float64')]

In [46]:
num_cols = ["n_second_good_colname", "n_second_bad_colname"]
cat_cols = ["c_first_good_colname", "c_first_bad_colname"]
df3 = ensure_column_types(df, num_cols, cat_cols)

In [47]:
df3

,c_first_good_colname,n_second_good_colname,c_first_bad_colname,n_second_bad_colname
0,cat1,1,7,3.0
1,cat2,3,8,2.2
2,cat1,5,9,5.6


In [48]:
from numpy import dtype

In [49]:
expected_types = [dtype('O'), dtype('float64'), dtype('O'), dtype('float64')]

In [50]:
expected_types

[dtype('O'), dtype('float64'), dtype('O'), dtype('float64')]

In [51]:
df3.dtypes.to_list()

[CategoricalDtype(categories=['cat1', 'cat2'], ordered=False),
 dtype('int64'),
 CategoricalDtype(categories=[7, 8, 9], ordered=False),
 dtype('float64')]

In [52]:
df3["n_second_good_colname"].dtype, df3["c_first_good_colname"].dtype

(dtype('int64'), CategoricalDtype(categories=['cat1', 'cat2'], ordered=False))

## Imputation

In [53]:
import pandas as pd
import numpy as np

In [54]:
imputation_sample = pd.DataFrame(
    {
        "i_id_col": [1, 2, 3, 4, 5],
        "n_num_col_complete": [4, 2, 6, 1, 0],
        "n_num_col_missing_not_listed": [4.0, np.nan, 2.1, 3.2, np.nan],
        "n_num_col_missing_for_mean": [2, np.nan, 5, np.nan, 8],
        "n_num_col_missing_for_zero": [np.nan, 9.3, -1.2, np.nan, np.nan],
        "c_cat_col_for_mostfreq": ["cat1", np.nan, "cat2", "cat1", "cat1"],
        "c_cat_col_for_unknown": ["cat1", "cat2", np.nan, "cat3", "cat2"],
    }
)

In [55]:
imputation_sample

,i_id_col,n_num_col_complete,n_num_col_missing_not_listed,n_num_col_missing_for_mean,n_num_col_missing_for_zero,c_cat_col_for_mostfreq,c_cat_col_for_unknown
0,1,4,4.0,2.0,NaN,cat1,cat1
1,2,2,NaN,NaN,9.3,NaN,cat2
2,3,6,2.1,5.0,-1.2,cat2,NaN
3,4,1,3.2,NaN,NaN,cat1,cat3
4,5,0,NaN,8.0,NaN,cat1,cat2


In [56]:
imputation_sample.iloc[1, 3]

nan

In [57]:
imputation_sample.iloc[0, 4]

nan

In [58]:
from ga4_mlops.pipelines.feature_engineering.nodes import fit_imputers, apply_imputers

In [59]:
expected_df = pd.DataFrame(
    {
        "i_id_col": [1, 2, 3, 4, 5],
        "n_num_col_complete": [4, 2, 6, 1, 0],
        "n_num_col_missing_not_listed": [4.0, np.nan, 2.1, 3.2, np.nan],
        "n_num_col_missing_for_mean": [2., 5., 5., 5., 8.],
        "n_num_col_missing_for_zero": [0.0, 9.3, -1.2, 0.0, 0.0],
        "c_cat_col_for_mostfreq": ["cat1", "cat1", "cat2", "cat1", "cat1"],
        "c_cat_col_for_unknown": ["cat1", "cat2", "UNKNOWN", "cat3", "cat2"],
    }
)

In [60]:
expected_df

,i_id_col,n_num_col_complete,n_num_col_missing_not_listed,n_num_col_missing_for_mean,n_num_col_missing_for_zero,c_cat_col_for_mostfreq,c_cat_col_for_unknown
0,1,4,4.0,2.0,0.0,cat1,cat1
1,2,2,NaN,5.0,9.3,cat1,cat2
2,3,6,2.1,5.0,-1.2,cat2,UNKNOWN
3,4,1,3.2,5.0,0.0,cat1,cat3
4,5,0,NaN,8.0,0.0,cat1,cat2


In [61]:
imputation_strategies = {
  "mean": ["n_num_col_missing_for_mean"],   # for numerical: replace NULLs with column mean
  "zero": ["n_num_col_missing_for_zero"],    # for numerical: replace NULLs with zeros
  "mostfreq": ["c_cat_col_for_mostfreq"],    # for categorical: replace NULLs with most frequent value
  "unknown": ["c_cat_col_for_unknown"],    # for categorical: replace NULLs with UNKNOWN token
}

In [62]:
imputers = fit_imputers(imputation_sample, imputation_strategies)

In [63]:
imputers

{'mean': SimpleImputer(),
 'zero': SimpleImputer(fill_value=0.0, strategy='constant'),
 'mostfreq': SimpleImputer(strategy='most_frequent'),
 'unknown': SimpleImputer(fill_value='UNKNOWN', strategy='constant')}

In [64]:
df = apply_imputers(imputation_sample, imputers)

In [65]:
df

,i_id_col,n_num_col_complete,n_num_col_missing_not_listed,n_num_col_missing_for_mean,n_num_col_missing_for_zero,c_cat_col_for_mostfreq,c_cat_col_for_unknown
0,1,4,4.0,2.0,0.0,cat1,cat1
1,2,2,NaN,5.0,9.3,cat1,cat2
2,3,6,2.1,5.0,-1.2,cat2,UNKNOWN
3,4,1,3.2,5.0,0.0,cat1,cat3
4,5,0,NaN,8.0,0.0,cat1,cat2


In [66]:
df.dtypes

i_id_col                          int64
n_num_col_complete                int64
n_num_col_missing_not_listed    float64
n_num_col_missing_for_mean      float64
n_num_col_missing_for_zero      float64
c_cat_col_for_mostfreq           object
c_cat_col_for_unknown            object
dtype: object

In [67]:
expected_df.dtypes

i_id_col                          int64
n_num_col_complete                int64
n_num_col_missing_not_listed    float64
n_num_col_missing_for_mean      float64
n_num_col_missing_for_zero      float64
c_cat_col_for_mostfreq           object
c_cat_col_for_unknown            object
dtype: object

In [68]:
df.equals(expected_df)

True

## Encoding

In [69]:
import pandas as pd

In [70]:
encoding_sample = pd.DataFrame(
    {
        "i_id_col": [1, 2, 3, 4, 5],
        "c_cat_col_for_binary": ["0", "0", "1", "0", "1"],
        "c_cat_col_for_onehot": ["cat1", "cat2", "cat3", "cat3", "cat2"],
        "c_cat_col_for_ordinal": ["cat3", "cat2", "cat1", "cat3", "cat2"],
    }
)

In [71]:
encoding_sample

,i_id_col,c_cat_col_for_binary,c_cat_col_for_onehot,c_cat_col_for_ordinal
0,1,0,cat1,cat3
1,2,0,cat2,cat2
2,3,1,cat3,cat1
3,4,0,cat3,cat3
4,5,1,cat2,cat2


In [72]:
encoder_types = {
    "binary": ["c_cat_col_for_binary"],   # for binary variables
    "onehot": ["c_cat_col_for_onehot"],   # one-hot encoding
    "ordinal": ["c_cat_col_for_ordinal"]   # integer encoding (ordinal, but order does not matter)
}

In [73]:
from ga4_mlops.pipelines.feature_engineering.nodes import fit_encoders, apply_encoders

In [74]:
encoders = fit_encoders(encoding_sample, encoder_types)

/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/category_encoders/one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():


In [75]:
df = apply_encoders(encoding_sample, encoders)

In [76]:
df

,i_id_col,c_cat_col_for_binary_0,c_cat_col_for_binary_1,c_cat_col_for_onehot_cat1,c_cat_col_for_onehot_cat2,c_cat_col_for_onehot_cat3,c_cat_col_for_ordinal
0,1,0,1,1,0,0,1
1,2,0,1,0,1,0,2
2,3,1,0,0,0,1,3
3,4,0,1,0,0,1,1
4,5,1,0,0,1,0,2


In [77]:
df.to_dict(orient="list")

{'i_id_col': [1, 2, 3, 4, 5],
 'c_cat_col_for_binary_0': [0, 0, 1, 0, 1],
 'c_cat_col_for_binary_1': [1, 1, 0, 1, 0],
 'c_cat_col_for_onehot_cat1': [1, 0, 0, 0, 0],
 'c_cat_col_for_onehot_cat2': [0, 1, 0, 0, 1],
 'c_cat_col_for_onehot_cat3': [0, 0, 1, 1, 0],
 'c_cat_col_for_ordinal': [1, 2, 3, 1, 2]}

In [78]:
expected_df = pd.DataFrame(
    {
        "i_id_col": [1, 2, 3, 4, 5],
        "c_cat_col_for_binary_0": [0, 0, 1, 0, 1],
        "c_cat_col_for_binary_1": [1, 1, 0, 1, 0],
        "c_cat_col_for_onehot_cat1": [1, 0, 0, 0, 0],
        "c_cat_col_for_onehot_cat2": [0, 1, 0, 0, 1],
        "c_cat_col_for_onehot_cat3": [0, 0, 1, 1, 0],
        "c_cat_col_for_ordinal": [1, 2, 3, 1, 2]
    }
)

In [79]:
df.equals(expected_df)

True

## Exclusion

In [80]:
import pandas as pd

In [81]:
exclusion_sample = pd.DataFrame(
    {
        "i_id_col": [1, 2, 3, 4, 5],
        "n_col_to_use": [3.2, 2.1, 9.8, 0.3, 3.1],
        "c_col_to_use_0": [0, 1, 0, 0, 0],
        "c_col_to_use_1": [1, 0, 1, 1, 1],
        "c_first_col_to_exclude": [1, 3, 2, 1, 2],
        "c_second_col_to_exclude_cat1": [1, 1, 0, 0, 0],
        "c_second_col_to_exclude_cat2": [0, 0, 1, 1, 0],
        "c_second_col_to_exclude_cat2": [0, 0, 0, 0, 1],
    }
)

In [82]:
exclusion_sample

,i_id_col,n_col_to_use,c_col_to_use_0,c_col_to_use_1,c_first_col_to_exclude,c_second_col_to_exclude_cat1,c_second_col_to_exclude_cat2
0,1,3.2,0,1,1,1,0
1,2,2.1,1,0,3,1,0
2,3,9.8,0,1,2,0,0
3,4,0.3,0,1,1,0,0
4,5,3.1,0,1,2,0,1


In [83]:
features_to_exclude = ["c_first_col_to_exclude", "c_second_col_to_exclude"]

In [84]:
from ga4_mlops.pipelines.feature_engineering.nodes import exclude_features

In [85]:
df = exclude_features(exclusion_sample, features_to_exclude)

In [86]:
df

,i_id_col,n_col_to_use,c_col_to_use_0,c_col_to_use_1,c_second_col_to_exclude_cat1,c_second_col_to_exclude_cat2
0,1,3.2,0,1,1,0
1,2,2.1,1,0,1,0
2,3,9.8,0,1,0,0
3,4,0.3,0,1,0,0
4,5,3.1,0,1,0,1


In [87]:
expected_df = pd.DataFrame(
    {
        "i_id_col": [1, 2, 3, 4, 5],
        "n_col_to_use": [3.2, 2.1, 9.8, 0.3, 3.1],
        "c_col_to_use_0": [0, 1, 0, 0, 0],
        "c_col_to_use_1": [1, 0, 1, 1, 1],
    }
)

In [88]:
expected_df

,i_id_col,n_col_to_use,c_col_to_use_0,c_col_to_use_1
0,1,3.2,0,1
1,2,2.1,1,0
2,3,9.8,0,1
3,4,0.3,0,1
4,5,3.1,0,1


In [89]:
df.equals(expected_df)

False

## Prediction

In [90]:
import numpy as np
import pandas as pd

In [91]:
abt_sample = pd.DataFrame(
    {
        "i_id": [1, 2, 3, 4, 5],
        "i_info_col": ["2022", "2021", "2022", "2022", "2021"],
        "n_num_col": [1.3, 5.6, 2.3, 7.6, 9.2],
        "c_cat_col": [1, 3, 3, 1, 2],
        "y_target_col": [1, 0, 0, 1, 0],
    }
)
raw_scores_sample = np.array([0.99, 0.02, 0.01, 0.98, 0.01])
calibrated_scores_sample = np.array([0.88, 0.42, 0.23, 0.84, 0.22])

In [92]:
abt_sample

,i_id,i_info_col,n_num_col,c_cat_col,y_target_col
0,1,2022,1.3,1,1
1,2,2021,5.6,3,0
2,3,2022,2.3,3,0
3,4,2022,7.6,1,1
4,5,2021,9.2,2,0


In [93]:
raw_scores_sample, calibrated_scores_sample

(array([0.99, 0.02, 0.01, 0.98, 0.01]), array([0.88, 0.42, 0.23, 0.84, 0.22]))

In [94]:
from ga4_mlops.pipelines.prediction.nodes import create_predictions

In [95]:
df = create_predictions(
    abt_sample,
    raw_scores_sample,
    calibrated_scores_sample,
    threshold=0.3,
    classify_on_calibrated=False
)

In [96]:
df

,i_id,i_info_col,y_raw_score,y_calibrated_score,y_predicted_label
0,1,2022,0.99,0.88,1
1,2,2021,0.02,0.42,0
2,3,2022,0.01,0.23,0
3,4,2022,0.98,0.84,1
4,5,2021,0.01,0.22,0


In [97]:
df = create_predictions(
    abt_sample,
    raw_scores_sample,
    calibrated_scores_sample,
    threshold=0.3,
    classify_on_calibrated=True
)

In [98]:
df

,i_id,i_info_col,y_raw_score,y_calibrated_score,y_predicted_label
0,1,2022,0.99,0.88,1
1,2,2021,0.02,0.42,1
2,3,2022,0.01,0.23,0
3,4,2022,0.98,0.84,1
4,5,2021,0.01,0.22,0


In [99]:
df.to_dict(orient="list")

{'i_id': [1, 2, 3, 4, 5],
 'i_info_col': ['2022', '2021', '2022', '2022', '2021'],
 'y_raw_score': [0.99, 0.02, 0.01, 0.98, 0.01],
 'y_calibrated_score': [0.88, 0.42, 0.23, 0.84, 0.22],
 'y_predicted_label': [1, 1, 0, 1, 0]}

## Explanation - sampling

In [100]:
import pandas as pd

In [101]:
explanation_sample = pd.DataFrame(
    {
        "i_id": list(range(20)),
        "y_target": [1]*5 + [0]*15
    }
)

In [102]:
explanation_sample

,i_id,y_target
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [103]:
from ga4_mlops.pipelines.explanation.nodes import sample_data

/Users/pchaberski/projects/gid-ml-framework/ga4-mlops/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [104]:
df = sample_data(explanation_sample, n_obs=8, seed=42)

In [105]:
df

,i_id,y_target
0,14,0
1,16,0
2,5,0
3,18,0
4,10,0
5,13,0
6,1,1
7,4,1


In [106]:
df.to_dict(orient="list")

{'i_id': [14, 16, 5, 18, 10, 13, 1, 4], 'y_target': [0, 0, 0, 0, 0, 0, 1, 1]}

## Ensure column types v2

In [107]:
import numpy as np
import pandas as pd

In [108]:
wrong_column_types_sample = pd.DataFrame(
        {
            "c_first_good_coltype": ["cat1", "cat2", "cat1"],
            "n_second_good_coltype": [1, 3, 5],
            "c_third_good_coltype": ["cat2", pd.NA, "cat3"],
            "n_fourth_good_coltype": [4.2, np.NaN, 2.9],
            "c_fifth_good_coltype": [pd.NA, pd.NA, pd.NA],
            "n_sixth_good_coltype": [np.NaN, np.NaN, np.NaN],
            "c_first_bad_coltype": [7, 8, 9],
            "c_second_bad_coltype": [4.1, 2.3, 8.9],
            "n_third_bad_coltype": ["3.0", "2.2", "5.6"],
            "n_fourth_bad_coltype": [4.2, pd.NA, 2.9],
            "n_fifth_bad_coltype": [3, pd.NA, 2],
            "c_sixth_bad_coltype": [np.NaN, np.NaN, np.NaN],
            "n_seventh_bad_coltype": [pd.NA, pd.NA, pd.NA],
        }
    )

In [109]:
wrong_column_types_sample

,c_first_good_coltype,n_second_good_coltype,c_third_good_coltype,n_fourth_good_coltype,c_fifth_good_coltype,n_sixth_good_coltype,c_first_bad_coltype,c_second_bad_coltype,n_third_bad_coltype,n_fourth_bad_coltype,n_fifth_bad_coltype,c_sixth_bad_coltype,n_seventh_bad_coltype
0,cat1,1,cat2,4.2,<NA>,NaN,7,4.1,3.0,4.2,3,NaN,<NA>
1,cat2,3,<NA>,NaN,<NA>,NaN,8,2.3,2.2,<NA>,<NA>,NaN,<NA>
2,cat1,5,cat3,2.9,<NA>,NaN,9,8.9,5.6,2.9,2,NaN,<NA>


In [110]:
wrong_column_types_sample.dtypes

c_first_good_coltype      object
n_second_good_coltype      int64
c_third_good_coltype      object
n_fourth_good_coltype    float64
c_fifth_good_coltype      object
n_sixth_good_coltype     float64
c_first_bad_coltype        int64
c_second_bad_coltype     float64
n_third_bad_coltype       object
n_fourth_bad_coltype      object
n_fifth_bad_coltype       object
c_sixth_bad_coltype      float64
n_seventh_bad_coltype     object
dtype: object

In [111]:
from ga4_mlops.pipelines.data_preparation_utils import extract_column_names, ensure_column_types

In [112]:
_, num_cols, cat_cols, _ = extract_column_names(wrong_column_types_sample)

In [113]:
df = ensure_column_types(wrong_column_types_sample, num_cols, cat_cols)

In [114]:
df

,c_first_good_coltype,n_second_good_coltype,c_third_good_coltype,n_fourth_good_coltype,c_fifth_good_coltype,n_sixth_good_coltype,c_first_bad_coltype,c_second_bad_coltype,n_third_bad_coltype,n_fourth_bad_coltype,n_fifth_bad_coltype,c_sixth_bad_coltype,n_seventh_bad_coltype
0,cat1,1,cat2,4.2,NaN,NaN,7,4.1,3.0,4.2,3.0,NaN,NaN
1,cat2,3,NaN,NaN,NaN,NaN,8,2.3,2.2,NaN,NaN,NaN,NaN
2,cat1,5,cat3,2.9,NaN,NaN,9,8.9,5.6,2.9,2.0,NaN,NaN


In [115]:
df.dtypes

c_first_good_coltype     category
n_second_good_coltype       int64
c_third_good_coltype     category
n_fourth_good_coltype     float64
c_fifth_good_coltype     category
n_sixth_good_coltype      float64
c_first_bad_coltype      category
c_second_bad_coltype     category
n_third_bad_coltype       float64
n_fourth_bad_coltype      float64
n_fifth_bad_coltype       float64
c_sixth_bad_coltype      category
n_seventh_bad_coltype     float64
dtype: object

In [116]:
df["c_fifth_good_coltype"]

0    NaN
1    NaN
2    NaN
Name: c_fifth_good_coltype, dtype: category
Categories (0, object): []

In [117]:
type(df["c_fifth_good_coltype"][0])

float

In [118]:
df["c_sixth_bad_coltype"]

Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


0   NaN
1   NaN
2   NaN
Name: c_sixth_bad_coltype, dtype: category
Categories (0, float64): []

In [119]:
type(df["c_sixth_bad_coltype"][0])

float

In [120]:
list(df.dtypes)

[CategoricalDtype(categories=['cat1', 'cat2'], ordered=False),
 dtype('int64'),
 CategoricalDtype(categories=['cat2', 'cat3'], ordered=False),
 dtype('float64'),
 CategoricalDtype(categories=[], ordered=False),
 dtype('float64'),
 CategoricalDtype(categories=[7, 8, 9], ordered=False),
 CategoricalDtype(categories=[2.3, 4.1, 8.9], ordered=False),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 CategoricalDtype(categories=[], ordered=False),
 dtype('float64')]